In [ ]:
import numpy as np
import json
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling2D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder



chatbot project version 0

In [ ]:
# import dataset
with open('/content/intents.json') as file:
  data=json.load(file)


training_sentences = []
training_labels=[]
labels=[]
responses = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])

num_classes=len(labels)
num_classes

8

In [ ]:
# label encoder
lbl_encoder=LabelEncoder()
lbl_encoder.fit(training_labels)

training_labels=lbl_encoder.transform(training_labels)
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [ ]:
# tok=cknization of dataset
vocab_size=1000
embedding_dim=16
max_len=200
oov_token="<00v>"

tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index

sequences=tokenizer.texts_to_sequences(training_sentences)
padded_sequences=pad_sequences(sequences, truncating="post", maxlen=max_len)



In [ ]:
# training neural network
model=Sequential()

model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))

model.add(GlobalAveragePooling1D())

model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
epochs=10
history=model.fit(padded_sequences, np.array(training_labels), epochs=epochs)




/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 620ms/step - accuracy: 0.0919 - loss: 2.0848
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0814 - loss: 2.0795
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0919 - loss: 2.0795
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0919 - loss: 2.0788
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0919 - loss: 2.0767
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0919 - loss: 2.0765 
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2143 - loss: 2.0732
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2143 - loss: 2.0719
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2143 - loss: 2.0721 
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2143 - loss: 2.0712


In [ ]:
# save trained model
model.save("chat.h5")

import pickle

with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as ecn_file:
  pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)




In [ ]:
# output package import
!pip install colorama
import colorama

colorama.init()

from colorama import Fore, Style, Back

import random



In [ ]:
# output


with open('/content/intents.json') as file:
  data = json.load(file)

def chat():
  model=load_model('chat.h5')

  with open('tokenizer.pickle', 'rb') as handle:
    tokenizer=pickle.load(handle)

  with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder=pickle.load(enc)

  max_len=20

  while True:
    print(Fore.LIGHTBLUE_EX + "User:" + Style.RESET_ALL, end="")
    inp = input()

    if isinstance(inp, str):
      input_text = inp.lower()
    else:
    # Handle the case when it's not a string
      print("Error: input_text is not a string")

    if inp.lower()=="quit":
      break

    result=model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post', maxlen=max_len))

    tag=lbl_encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
      if i['tag'] == tag:
        print(Fore.GREEN+ "ChatBot: " + Style.RESET_ALL, np.random.choice(i['responses']))


print(Fore.YELLOW+"Start message " + Style.RESET_ALL)

chat()









Start message 


User:is anyone here?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
ChatBot:  Tell me how can assist you
User:bye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
ChatBot:  Tell me how can assist you
User:

KeyboardInterrupt: Interrupted by user